In [28]:
import importlib
import autovc as autovc
import numpy as np
import pickle
import tensorflow as tf

## Experiment with meta data from autovc repo

In [29]:
with open('metadata.pkl', 'rb') as f:
    data = pickle.load(f)
# data = np.array(data)
mel_specs, sp_emb = [], [] 
for d in data:
    sp_emb.append(d[1])
    print(d[2].shape)
    mel_specs.append(d[2])
mel_specs = np.array(mel_specs)
sp_emb = np.array(sp_emb).astype('float32')

(90, 80)
(89, 80)
(75, 80)
(109, 80)
ipykernel_launcher:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [30]:
sp_emb.shape

(4, 256)

In [31]:
mel_specs.shape

(4,)

In [49]:
importlib.reload(autovc)
padded_mel_specs = tf.keras.preprocessing.sequence.pad_sequences(mel_specs, padding="post", dtype='float32',value=-1.0, maxlen=128, truncating="post")
model = autovc.AutoVC(time_dim=128, dim_emb=256, dim_pre=512, dim_neck=16)
model.compile(optimizer="Adam", metrics=["loss"], loss = [tf.keras.losses.MSE, tf.keras.losses.MSE])
# mel_output, mel_output_postnet = model((padded_mel_specs, sp_emb, sp_emb))

In [52]:
# model.fit((padded_mel_specs, sp_emb, sp_emb), (padded_mel_specs, padded_mel_specs),epochs=1, batch_size=1)

## Test Encoder with Functional API 

In [86]:
importlib.reload(autovc)
encoder = autovc.Encoder()
codes = encoder(padded_mel_specs, sp_emb)

test (4, 128, 80)
(4, 512, 32)


In [76]:
codes.shape

TensorShape([16, 4, 32])

In [85]:
tf.keras.layers.UpSampling1D(32)(codes).shape

TensorShape([16, 128, 32])

## Test UpSampling with Functional API

In [88]:
importlib.reload(autovc)
upsample = autovc.UpSampling()
output = upsample(codes, sp_emb)

## Test Decoder with Functional API 

In [95]:
importlib.reload(autovc)
decoder = autovc.Decoder()
mel_decoder = decoder(output)

In [94]:
mel_decoder.shape

TensorShape([4, 128, 80])

## Test Posnet with Functional API

In [96]:
importlib.reload(autovc)
postnet = autovc.PostNet()
mel_postnet = postnet(mel_decoder)

In [97]:
mel_postnet.shape

TensorShape([4, 128, 80])